In [15]:
!pip install pyspark
!pip install spark-nlp==2.0.1
!pip install pandas

In [0]:
import pandas as pd
pd.set_option('max_colwidth', 800)

In [0]:
from pyspark.sql import SparkSession

spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "JohnSnowLabs:spark-nlp:1.8.2") \
        .getOrCreate()

In [21]:
path = 'data/'
df = spark.read.format('csv').option('header', 'true').load(path)
df.limit(5).toPandas()

,_c0,id,created_at,retweet_count,user_screen_name,user_followers_count,user_location,Hashtags,text,Sentiment,SentimentClass
0,0,1259505474501849088,2020-05-10 15:28:00,56,carrierkoo,264,None,[],RT @samirasawlani: Govt of Zambia orders temporary closure of border with Tanzania from tomorrow. This after 76 Covid-19 cases were reporte…,0.0,Neutral
1,1,1259501817655853058,2020-05-10 15:13:28,3,Mwanachile2,33,None,[],"RT @mohzambia: Zambia has recorded 15 new cases of COVID-19 in the last 24hrs. 14 in Lusaka, 1 in Chilanga, 5 New Recoveries COVID-19 #48…",0.0909090909090909,Positive
2,2,1259495706496323591,2020-05-10 14:49:11,1,fmdien,986,Earth,[],"RT @immortal_mia: @Mujajati_a More 'children' have died from Malaria during Covid-19 'lockdown' (qtr4.2019/qtr1.2020), than Covid-19, TB, M…",0.5,Positive
3,3,1259494947402788869,2020-05-10 14:46:10,1,AfricanPostMag,934,Ghana,[],RT @zadama24: COVID-19 : Zambia records 15 new cases of COVID-19.,0.13636363636363635,Positive
4,4,1259494621899632640,2020-05-10 14:44:52,1,zadama24,61547,Lusaka,[],COVID-19 : Zambia records 15 new cases of COVID-19.,0.13636363636363635,Positive


In [23]:
from pyspark.sql.functions import split, explode, desc

dfWords = df.select(explode(split('text', '\\s+')).alias('word')) \
                    .groupBy('word').count().orderBy(desc('word'))
dfWords.printSchema()

root
 |-- word: string (nullable = true)
 |-- count: long (nullable = false)



In [24]:
dfWords.orderBy(desc('count')).limit(5).toPandas()

,word,count
0,the,536
1,in,458
2,of,421
3,Zambia,354
4,to,310


In [25]:
from com.johnsnowlabs.nlp.pretrained.pipeline.en import BasicPipeline as bp

dfAnnotated = bp.annotate(df, 'text')
dfAnnotated.printSchema()

IllegalArgumentException: ignored